### CODE ADAPTED FROM https://towardsdatascience.com/web-scraping-scraping-table-data-1665b6b2271c

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np

### Scraping 2006 to 2018 data

In [2]:
url = "https://www.cdc.gov/drugoverdose/rxrate-maps/county2006.html"

column_names = ["Year", "County", "State", "County FIPS Code", "Opioid Dispensing Rate per 100"]

opioidrate_06_18_df = pd.DataFrame(columns = column_names)

for year in range(2006, 2019):
    url = re.sub(r"\d+", str(year), url)
    
    
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text

    # Parse HTML code for the entire site
    soup = BeautifulSoup(html_content, "lxml")
    opioid_table = soup.find_all("table")
    
    # Lets go ahead and scrape first table with HTML code gdp[0]
    table = opioid_table[0]
    # the head will form our column names
    body = table.find_all("tr")
    # Head values (Column names) are the first items of the body list
#     head = body[0]
    
    body_rows = body[1:] # All other items becomes the rest of the rows
    
#     # Declare empty list to keep Columns names
#     headings = []
#     for item in head.find_all("th"): # loop through all th elements
#         # convert the th elements to text and strip "\n"
#         item = (item.text).rstrip("\n")
#         # append the clean column name to headings
#         headings.append(item)
    
    all_rows = [] # will be a list for list for all rows
    for row_num in range(len(body_rows)): # A row at a time
        row = [] # this will old entries for one row
        for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
            # row_item.text removes the tags from the entries
            # the following regex is to remove \xa0 and \n and comma from row_item.text
            # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
            aa = re.sub("(\xa0)|(\n)|","",row_item.text)
            #append aa to row - note one row entry is being appended
            row.append(aa)
        # append one row to all_rows
        row.insert(0, year)
        all_rows.append(row)
    
    print(type(all_rows))
    year_df = pd.DataFrame(data=all_rows, columns=opioidrate_06_18_df.columns)
    opioidrate_06_18_df = opioidrate_06_18_df.append(year_df)

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [3]:
# Removing state abbreviation from County column
opioidrate_06_18_df['County'] = opioidrate_06_18_df['County'].str.split(',').str[0]

In [4]:
# Making County column title case
opioidrate_06_18_df['County'] = opioidrate_06_18_df['County'].str.title()

In [5]:
opioidrate_06_18_df

,Year,County,State,County FIPS Code,Opioid Dispensing Rate per 100
0,2006,Aleutians East,AK,02013,–
1,2006,Aleutians West,AK,02016,–
2,2006,Anchorage,AK,02020,71.5
3,2006,Bethel,AK,02050,–
4,2006,Bristol Bay,AK,02060,–
...,...,...,...,...,...
2876,2018,Sweetwater,WY,56037,79.2
2877,2018,Teton,WY,56039,42.6
2878,2018,Uinta,WY,56041,95.9
2879,2018,Washakie,WY,56043,75.0


### Scraping 2019 data

In [6]:
url = "https://www.cdc.gov/drugoverdose/rxrate-maps/county2019.html"


# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")
opioid_table = soup.find_all("table")

# Lets go ahead and scrape first table with HTML code gdp[0]
table = opioid_table[0]
# the head will form our column names
body = table.find_all("tr")
# Head values (Column names) are the first items of the body list
#     head = body[0]

body_rows = body[1:] # All other items becomes the rest of the rows

all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    row.insert(0, 2019)
    all_rows.append(row)

column_names = ["Year", "State", "County", "County FIPS Code", "Opioid Dispensing Rate per 100"]

opioidrate_19_df = pd.DataFrame(data=all_rows, columns = column_names)

In [7]:
# Reordering the columns
opioidrate_19_df = opioidrate_19_df[["Year", "County", "State", "County FIPS Code", "Opioid Dispensing Rate per 100"]]

### Reading in 2020 data

In [8]:
opioidrate_2020 = pd.read_csv("opioidrate_2020.txt", sep = '\t')
opioidrate_2020.insert(0, 'Year', 2020)
opioidrate_2020['County'] = opioidrate_2020['County'].str.title()
opioidrate_2020 = opioidrate_2020[["Year", "County", "State", "County FIPS Code", "Opioid Dispensing Rate per 100"]]

### Combining 2006-18, 2019, and 2020 data

In [9]:
opioid_dispense = pd.concat([opioidrate_06_18_df, opioidrate_19_df, opioidrate_2020])

### Cleaning 2017 data

In [10]:
# Switching the values for County and State columns
opioid_dispense.loc[(opioid_dispense['Year'] == 2017), ['County', 'State']]=opioid_dispense.loc[(opioid_dispense['Year'] == 2017), ['State', 'County']].values

In [11]:
# Making County column title case
opioid_dispense.loc[(opioid_dispense['Year'] == 2017), 'County'] = opioid_dispense.loc[(opioid_dispense['Year'] == 2017), 'County'].str.title()

In [12]:
# Making state column uppercase
opioid_dispense.loc[(opioid_dispense['Year'] == 2017), 'State'] = opioid_dispense.loc[(opioid_dispense['Year'] == 2017), 'State'].str.upper()

### More cleaning

In [13]:
# renaming columns to single terms
opioid_dispense = opioid_dispense.rename(columns={'Year': 'Year', 'County': 'County', 'State': 'State', 'County FIPS Code': 'FIPS', 'Opioid Dispensing Rate per 100': 'dispenserate100'})

In [14]:
opioid_dispense.dtypes

Year               object
County             object
State              object
FIPS               object
dispenserate100    object
dtype: object

In [15]:
# replace all – with NaN
opioid_dispense['dispenserate100'] = opioid_dispense['dispenserate100'].replace('–', np.NaN)

In [16]:
# change data type
opioid_dispense.astype({'Year': 'int64', 'County': 'object', 'State': 'object', 'FIPS': 'int64', 'dispenserate100': 'float64'}).dtypes

Year                 int64
County              object
State               object
FIPS                 int64
dispenserate100    float64
dtype: object

In [17]:
# write to csv
opioid_dispense.to_csv('opioid_dispense.csv')